In [1]:
import numpy
import pandas as pd

In [2]:
df = pd.read_csv("Son-Veri-Seti.csv",header=0,infer_datetime_format=True)
df.shape

(1442, 10)

In [3]:
df.head()

,Unnamed: 0,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3,Sub_metering_4,Result
0,0,1209,34,93552,5180,0,546,4926,14680,0
1,1,3390,226,345725,14398,2033,4187,13341,36946,1
2,2,2203,161,347373,9247,1063,2621,14018,19028,0
3,3,1666,150,348479,7094,839,7602,6197,13131,1
4,4,2225,160,348923,9312,0,2648,14063,20384,0


In [4]:
df = df.iloc[:,1:10]
df.head()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3,Sub_metering_4,Result
0,1209,34,93552,5180,0,546,4926,14680,0
1,3390,226,345725,14398,2033,4187,13341,36946,1
2,2203,161,347373,9247,1063,2621,14018,19028,0
3,1666,150,348479,7094,839,7602,6197,13131,1
4,2225,160,348923,9312,0,2648,14063,20384,0


In [5]:
df.tail()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3,Sub_metering_4,Result
1437,2041,142,345883,8660,4855,2110,10136,16924,0
1438,1577,137,346428,6731,1871,458,7611,16352,0
1439,1796,132,345644,7559,1096,2848,12224,13769,0
1440,1431,116,347812,6004,1076,426,5072,17278,0
1441,1488,120,303487,6259,1080,385,9989,13347,0


In [6]:
dataset=numpy.array(df)
dataset

array([[  1209,     34,  93552, ...,   4926,  14680,      0],
       [  3390,    226, 345725, ...,  13341,  36946,      1],
       [  2203,    161, 347373, ...,  14018,  19028,      0],
       ...,
       [  1796,    132, 345644, ...,  12224,  13769,      0],
       [  1431,    116, 347812, ...,   5072,  17278,      0],
       [  1488,    120, 303487, ...,   9989,  13347,      0]], dtype=int64)

# 1- Parti Boyutunu ve Dönem Sayısı Nasıl Ayarlanır

In [7]:
# Epochs ve batch size'ı ızgarada aramak için scikit-learn'i kullanalım.
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

In [8]:
# KerasClassifier için gerekli olan modeli oluşturma 
def create_model():
	# Model oluşturma
	model = Sequential()
	model.add(Dense(12, input_dim=8, activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
	# Modeli derleme
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model
# Tekrarlanabilirlik için rastgele tohumu ayarladık.
seed = 7
numpy.random.seed(seed)
# Giriş (X) ve çıkış (Y) değişkenlerine böldük
X = dataset[:,0:8]
Y = dataset[:,8:]
# model oluşturma
model = KerasClassifier(build_fn=create_model, verbose=0)
# Izgara arama parametrelerini tanımladık.
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)
# Sonuçları özetleyelim.
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.656044 using {'batch_size': 20, 'epochs': 50}
0.655351 (0.018112) with: {'batch_size': 10, 'epochs': 10}
0.550488 (0.139483) with: {'batch_size': 10, 'epochs': 50}
0.583276 (0.116954) with: {'batch_size': 10, 'epochs': 100}
0.463155 (0.121623) with: {'batch_size': 20, 'epochs': 10}
0.656044 (0.017140) with: {'batch_size': 20, 'epochs': 50}
0.456456 (0.149260) with: {'batch_size': 20, 'epochs': 100}
0.551284 (0.057541) with: {'batch_size': 40, 'epochs': 10}
0.539530 (0.076402) with: {'batch_size': 40, 'epochs': 50}
0.465465 (0.143001) with: {'batch_size': 40, 'epochs': 100}
0.561765 (0.054016) with: {'batch_size': 60, 'epochs': 10}
0.653962 (0.016997) with: {'batch_size': 60, 'epochs': 50}
0.647032 (0.026794) with: {'batch_size': 60, 'epochs': 100}
0.500690 (0.081552) with: {'batch_size': 80, 'epochs': 10}
0.443151 (0.034043) with: {'batch_size': 80, 'epochs': 50}
0.440982 (0.133814) with: {'batch_size': 80, 'epochs': 100}
0.559644 (0.055392) with: {'batch_size': 100, 'epochs': 

# 2- Eğitim Optimizasyon Algoritmasını Ayarlama

In [9]:
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

In [10]:
# KerasClassifier için gerekli olan modeli oluşturma
def create_model(optimizer='adam'):
	# Model oluşturma
	model = Sequential()
	model.add(Dense(12, input_dim=8, activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
	# Modeli derleme
	model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
	return model
# Tekrarlanabilirlik için rastgele tohumu ayarladık.
seed = 7
numpy.random.seed(seed)
# Giriş (X) ve çıkış (Y) değişkenlerine böldük
X = dataset[:,0:8]
Y = dataset[:,8:]
# Model oluşturma
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)
# Izgara arama parametrelerini tanımladık.
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
param_grid = dict(optimizer=optimizer)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)
# Sonuçları özetleyelim.
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.654655 using {'optimizer': 'SGD'}
0.654655 (0.016017) with: {'optimizer': 'SGD'}
0.654655 (0.016017) with: {'optimizer': 'RMSprop'}
0.539582 (0.054928) with: {'optimizer': 'Adagrad'}
0.543544 (0.149260) with: {'optimizer': 'Adadelta'}
0.545854 (0.149813) with: {'optimizer': 'Adam'}
0.524371 (0.141967) with: {'optimizer': 'Adamax'}
0.566644 (0.140475) with: {'optimizer': 'Nadam'}


# 3- Öğrenme Hızını ve Momentum Ayarlama

In [11]:
# Öğrenme hızını ve momentumu ızgarada aramak için scikit-learn'i kullanalım
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import SGD

In [12]:
# KerasClassifier için gerekli olan modeli oluşturma
def create_model(learn_rate=0.01, momentum=0):
	# Model oluşturma
	model = Sequential()
	model.add(Dense(12, input_dim=8, activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
	# Modeli derleme
	optimizer = SGD(lr=learn_rate, momentum=momentum)
	model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
	return model
# Tekrarlanabilirlik için rastgele tohumu ayarladık.
seed = 7
numpy.random.seed(seed)
# Giriş (X) ve çıkış (Y) değişkenlerine böldük
X = dataset[:,0:8]
Y = dataset[:,8:]
# Model oluşturma
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)
# Izgara arama parametrelerini tanımladık.
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
param_grid = dict(learn_rate=learn_rate, momentum=momentum)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)
# Sonuçları özetleyelim.
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.654655 using {'learn_rate': 0.001, 'momentum': 0.0}
0.654655 (0.016017) with: {'learn_rate': 0.001, 'momentum': 0.0}
0.654655 (0.016017) with: {'learn_rate': 0.001, 'momentum': 0.2}
0.654655 (0.016017) with: {'learn_rate': 0.001, 'momentum': 0.4}
0.654655 (0.016017) with: {'learn_rate': 0.001, 'momentum': 0.6}
0.654655 (0.016017) with: {'learn_rate': 0.001, 'momentum': 0.8}
0.654655 (0.016017) with: {'learn_rate': 0.001, 'momentum': 0.9}
0.654655 (0.016017) with: {'learn_rate': 0.01, 'momentum': 0.0}
0.654655 (0.016017) with: {'learn_rate': 0.01, 'momentum': 0.2}
0.654655 (0.016017) with: {'learn_rate': 0.01, 'momentum': 0.4}
0.654655 (0.016017) with: {'learn_rate': 0.01, 'momentum': 0.6}
0.654655 (0.016017) with: {'learn_rate': 0.01, 'momentum': 0.8}
0.654655 (0.016017) with: {'learn_rate': 0.01, 'momentum': 0.9}
0.654655 (0.016017) with: {'learn_rate': 0.1, 'momentum': 0.0}
0.654655 (0.016017) with: {'learn_rate': 0.1, 'momentum': 0.2}
0.654655 (0.016017) with: {'learn_rate':

# 4- Ağ Ağırlığı Başlatma Nasıl Ayarlanır

In [13]:
# Ağırlık başlatmayı ızgarada aramak için scikit-learn'i kullanalım.
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

In [14]:
# KerasClassifier için gerekli olan modeli oluşturma
def create_model(init_mode='uniform'):
	# Model oluşturma
	model = Sequential()
	model.add(Dense(12, input_dim=8, kernel_initializer=init_mode, activation='relu'))
	model.add(Dense(1, kernel_initializer=init_mode, activation='sigmoid'))
	# Modeli derleme
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model
# Tekrarlanabilirlik için rastgele tohumu ayarladık.
seed = 7
numpy.random.seed(seed)
# Giriş (X) ve çıkış (Y) değişkenlerine böldük
X = dataset[:,0:8]
Y = dataset[:,8]
# Model oluşturma
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)
# Izgara arama parametrelerini tanımladık.
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
param_grid = dict(init_mode=init_mode)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)
# Sonuçları özetleyelim.
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.657427 using {'init_mode': 'normal'}
0.654655 (0.016017) with: {'init_mode': 'uniform'}
0.541696 (0.147402) with: {'init_mode': 'lecun_uniform'}
0.657427 (0.012101) with: {'init_mode': 'normal'}
0.654655 (0.016017) with: {'init_mode': 'zero'}
0.653266 (0.015068) with: {'init_mode': 'glorot_normal'}
0.566644 (0.140475) with: {'init_mode': 'glorot_uniform'}
0.505543 (0.115556) with: {'init_mode': 'he_normal'}
0.653269 (0.015197) with: {'init_mode': 'he_uniform'}


# 5- Nöron Aktivasyon Fonksiyonu Nasıl Ayarlanır

In [15]:
# Aktivasyon işlevini ızgarada aramak için scikit-learn'i kullanın
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

In [16]:
# KerasClassifier için gerekli olan modeli oluşturma
def create_model(activation='relu'):
	# Model oluşturma
	model = Sequential()
	model.add(Dense(12, input_dim=8, kernel_initializer='uniform', activation=activation))
	model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
	# Modeli derleme
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model
# Tekrarlanabilirlik için rastgele tohumu ayarladık.
seed = 7
numpy.random.seed(seed)
# Giriş (X) ve çıkış (Y) değişkenlerine böldük
X = dataset[:,0:8]
Y = dataset[:,8]
# Model oluşturma
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)
# Izgara arama parametrelerini tanımladık.
activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
param_grid = dict(activation=activation)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)
# Sonuçları özetleyelim.
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.654655 using {'activation': 'softmax'}
0.654655 (0.016017) with: {'activation': 'softmax'}
0.654655 (0.016017) with: {'activation': 'softplus'}
0.654655 (0.016017) with: {'activation': 'softsign'}
0.654655 (0.016017) with: {'activation': 'relu'}
0.654655 (0.016017) with: {'activation': 'tanh'}
0.654655 (0.016017) with: {'activation': 'sigmoid'}
0.654655 (0.016017) with: {'activation': 'hard_sigmoid'}
0.545161 (0.149397) with: {'activation': 'linear'}


# 6- Bırakma Düzenlemesini Ayarlama

In [17]:
# Bırakma oranını ızgarada aramak için scikit-learn'i kullanalım.
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.constraints import maxnorm

In [18]:
# KerasClassifier için gerekli olan modeli oluşturma
def create_model(dropout_rate=0.0, weight_constraint=0):
	# Model oluşturma
	model = Sequential()
	model.add(Dense(12, input_dim=8, kernel_initializer='uniform', activation='linear', kernel_constraint=maxnorm(weight_constraint)))
	model.add(Dropout(dropout_rate))
	model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
	# Modeli derleme
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model
# Tekrarlanabilirlik için rastgele tohumu ayarladık.
seed = 7
numpy.random.seed(seed)
# Giriş (X) ve çıkış (Y) değişkenlerine böldük
X = dataset[:,0:8]
Y = dataset[:,8]
# Model oluşturma
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)
#Izgara arama parametrelerini tanımladık.
weight_constraint = [1, 2, 3, 4, 5]
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
param_grid = dict(dropout_rate=dropout_rate, weight_constraint=weight_constraint)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)
# Sonuçları özetleyelim.
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.655348 using {'dropout_rate': 0.4, 'weight_constraint': 5}
0.603373 (0.061677) with: {'dropout_rate': 0.0, 'weight_constraint': 1}
0.622084 (0.062072) with: {'dropout_rate': 0.0, 'weight_constraint': 2}
0.514669 (0.120426) with: {'dropout_rate': 0.0, 'weight_constraint': 3}
0.629655 (0.030099) with: {'dropout_rate': 0.0, 'weight_constraint': 4}
0.654655 (0.016017) with: {'dropout_rate': 0.0, 'weight_constraint': 5}
0.654655 (0.014606) with: {'dropout_rate': 0.1, 'weight_constraint': 1}
0.654655 (0.016017) with: {'dropout_rate': 0.1, 'weight_constraint': 2}
0.433356 (0.140475) with: {'dropout_rate': 0.1, 'weight_constraint': 3}
0.654655 (0.016017) with: {'dropout_rate': 0.1, 'weight_constraint': 4}
0.550481 (0.146479) with: {'dropout_rate': 0.1, 'weight_constraint': 5}
0.654655 (0.016017) with: {'dropout_rate': 0.2, 'weight_constraint': 1}
0.654655 (0.016017) with: {'dropout_rate': 0.2, 'weight_constraint': 2}
0.608224 (0.081673) with: {'dropout_rate': 0.2, 'weight_constraint': 

# 7- Gizli Katmandaki Nöron Sayısı Nasıl Ayarlanır

In [19]:
# Nöron sayısını ızgarada aramak için scikit-learn'i kullanalım.
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.constraints import maxnorm

In [20]:
# KerasClassifier için gerekli olan modeli oluşturma
def create_model(neurons=1):
	# Model oluşturma
	model = Sequential()
	model.add(Dense(neurons, input_dim=8, kernel_initializer='uniform', activation='linear', kernel_constraint=maxnorm(4)))
	model.add(Dropout(0.2))
	model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
	# Modeli derleme
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model
# Tekrarlanabilirlik için rastgele tohumu ayarladık.
seed = 7
numpy.random.seed(seed)
# Giriş (X) ve çıkış (Y) değişkenlerine böldük
X = dataset[:,0:8]
Y = dataset[:,8]
# Model oluşturma
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)
# Izgara arama parametrelerini tanımladık.
neurons = [1, 5, 10, 15, 20, 25, 30]
param_grid = dict(neurons=neurons)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)
# Sonuçları özetleyelim.
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.654655 using {'neurons': 1}
0.654655 (0.016017) with: {'neurons': 1}
0.654655 (0.016017) with: {'neurons': 5}
0.458310 (0.146744) with: {'neurons': 10}
0.494340 (0.135699) with: {'neurons': 15}
0.454158 (0.134786) with: {'neurons': 20}
0.642874 (0.032673) with: {'neurons': 25}
0.549794 (0.140460) with: {'neurons': 30}
